# GNN Modeling

## Setting Up On Externel Environments

In [1]:
%%capture

#NOTE: use conda_python3 environment

# install pytorch
!pip3 install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio==0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

# install pytorch geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
    
# install pytorch lightning
!pip install pytorch-lightning
  
# install wandb
!pip install wandb

## Data and Set up

In [2]:
import numpy as np
import pandas as pd
import json

from tqdm import tqdm

np.random.seed(314159) # set random seed

import torch
import pytorch_lightning as pl

from torch_geometric.data import Data

import wandb

In [3]:
graph_used = 'know' # 'know' or 'phys': the PPI network to use

In [4]:
# read dataset
node_data_path = f'data/{graph_used}_graph/final_nodeonly_node_data_{graph_used}_v1.csv'
node_dataset = pd.read_csv(node_data_path, index_col=0)

# assert that all genes in dataset are unique
assert(node_dataset.index.duplicated().sum() == 0)

# create mapping to 0-based index map
genes = node_dataset.index.to_numpy()
gene_id_dict = {ensembl: idx for idx, ensembl in enumerate(genes)}

# map ID's in node dataset
myID = node_dataset.index.map(gene_id_dict).rename('myID')
node_dataset.insert(loc=0, column='myID', value=myID)
node_dataset = node_dataset.reset_index().set_index('myID')
node_dataset.drop(columns=['ensembl'], inplace=True)

In [5]:
# PROCESS LABELS

# read labels
labels = pd.read_csv(f'data/{graph_used}_graph/labels_thres0,02_trials_{graph_used}.csv', index_col=0)

# drop label columns from node_dataset
label_cols_drop = ['gda_score', 'gda_score_thres0.01', 'gda_score_thres0.02', 
              'gda_score_thres0.03', 'gda_score_thres0.04', 'gda_score_thres0.05']
node_dataset.drop(columns=label_cols_drop, inplace=True)

# map Ensmebl index to myID in labels dataset
myID = labels.index.map(gene_id_dict).rename('myID')
labels.insert(loc=0, column='myID', value=myID)
labels = labels.reset_index().set_index('myID')
labels.drop(columns=['ensembl'], inplace=True)

# convert type
num_label_trials = 100
for label_col in [f'label_{i}' for i in range(num_label_trials)]:
    labels[label_col] = labels[label_col].astype('Int32')

print('\ndistribution of labels')
print(labels['label_0'].value_counts())


distribution of labels
0    778
1    778
Name: label_0, dtype: Int64


In [6]:
node_dataset

hpa_0     hpa_1     hpa_2     hpa_3     hpa_4     hpa_5     hpa_6  \
myID                                                                          
0      0.443505  0.313574  0.430307 -0.285337 -0.351412  1.545059 -0.450936   
1      0.532289  0.789331 -0.122159  0.019000  1.427162 -0.065659  0.001669   
2      1.605229  0.936681  0.345886  0.183699 -1.001764 -0.846346  0.550995   
3      0.467196  0.690415 -0.066723  0.335478  1.017124  0.029579 -0.131714   
4      0.956718  0.202132 -0.490047 -0.524460  0.467132 -0.207775 -0.494652   
...         ...       ...       ...       ...       ...       ...       ...   
14849  1.436813 -1.455513  0.510199  0.224208  0.323736 -0.171141 -0.060697   
14850  1.140170 -1.162868  0.551409  0.127004  0.119115 -0.120434 -0.421260   
14851  0.782655 -0.057857 -0.397211  0.017498 -0.135123 -0.248882 -0.438176   
14852  1.042306  0.183392 -0.327697  0.065425 -0.459893  0.507171 -0.610148   
14853  0.553160  0.068709  0.013208  0.760541  0.592771 -0.094544  0.326136   

          hpa_7     hpa_8     hpa_9  ...    hpa_97    hpa_98    hpa_99  \
myID                                 ...                                 
0     -0.444200 -0.116774  0.183057  ...  0.001834  0.551479  0.897330   
1      0.386983 -0.028900  0.077691  ...  0.125478  0.140488  0.071253   
2      0.368205  0.202731  1.163779  ...  0.087994  0.149311 -0.057571   
3      0.212387  0.075120 -0.138351  ... -0.021957 -0.046296 -0.021106   
4      0.192615  0.077836 -0.134145  ...  0.006421 -0.016449  0.012585   
...         ...       ...       ...  ...       ...       ...       ...   
14849  0.088626  0.103071  0.000213  ... -0.005253 -0.000186  0.025215   
14850  0.798186  0.041917 -0.019688  ... -0.017310 -0.015128  0.031892   
14851  0.113363 -0.295955 -0.058939  ...  0.007004  0.065422 -0.036708   
14852  0.067066  0.368216 -0.383773  ...  0.194201  0.060288 -0.049898   
14853 -0.636445  0.047971 -0.211923  ...  0.004442  0.011485 -0.002870   

       nx_tissue_rna_lung  nx_sing_cell_type_mucus_secreting_cells  \
myID                                                                 
0               11.006867                                -0.105024   
1               -0.000111                                -0.105024   
2               -0.031058                                -0.086940   
3               -0.469481                                -0.073416   
4               -0.634534                                -0.105024   
...                   ...                                      ...   
14849           -0.603586                                -0.095432   
14850           -0.175480                                -0.095746   
14851           -0.325059                                -0.087883   
14852            2.001158                                -0.073730   
14853           -0.098111                                -0.057218   

       nih_ssm_in_cohort  nih_ssm_across_gdc  nih_cnv_gain  nih_cnv_loss  \
myID                                                                       
0               0.049383            0.031607      0.042885      0.066277   
1               0.001764            0.009436      0.048733      0.025341   
2               0.005291            0.008822      0.068226      0.019493   
3               0.031746            0.015036      0.038986      0.052632   
4               0.010582            0.010817      0.066277      0.011696   
...                  ...                 ...           ...           ...   
14849           0.024691            0.023168      0.033138      0.011696   
14850           0.014109            0.013195      0.033138      0.015595   
14851           0.022928            0.011891      0.035088      0.048733   
14852           0.012346            0.011047      0.064327      0.023392   
14853           0.044092            0.035827      0.011696      0.046784   

       nih_tot_mutations  
myID                      
0                     29  
1                      1  
2          

In [7]:
num_classes = 2
num_features = len(node_dataset.columns)

In [8]:
# load edge list
edge_list_path = f'data/{graph_used}_graph/final_edge_list_{graph_used}_features_v1.csv' # USING KNOWLEDGE GRAPH

edge_list = pd.read_csv(edge_list_path)

# map edge list
edge_list.gene1 = edge_list.gene1.map(gene_id_dict)
edge_list.gene2 = edge_list.gene2.map(gene_id_dict)

# scale edge features appropriately (they take values in the range 0-1000)
edge_feat_cols = edge_list.columns[2:].to_numpy()
edge_list[edge_feat_cols] /= 1000

edge_list

gene1  gene2  neighborhood  fusion  cooccurence  coexpression  \
0          839   8988           0.0     0.0          0.0         0.054   
1         4109   8988           0.0     0.0          0.0         0.053   
2        12757   8988           0.0     0.0          0.0         0.088   
3         9850   8988           0.0     0.0          0.0         0.000   
4         6690   8988           0.0     0.0          0.0         0.000   
...        ...    ...           ...     ...          ...           ...   
4405636   7841   7843           0.0     0.0          0.0         0.771   
4405637   7847   7843           0.0     0.0          0.0         0.647   
4405638   7842   7843           0.0     0.0          0.0         0.523   
4405639   7844   7843           0.0     0.0          0.0         0.523   
4405640   7846   7843           0.0     0.0          0.0         0.551   

         experimental  database  textmining  combined_score  
0               0.000       0.0       0.144           0.155  
1               0.000       0.0       0.246           0.255  
2               0.099       0.0       0.053           0.153  
3               0.000       0.0       0.206           0.205  
4               0.000       0.0       0.176           0.175  
...               ...       ...         ...             ...  
4405636         0.000       0.0       0.000           0.771  
4405637         0.000       0.0       0.000           0.647  
4405638         0.000       0.0       0.000           0.523  
4405639         0.000       0.0       0.000           0.523  
4405640         0.000       0.0       0.000           0.551  

[4405641 rows x 10 columns]

In [9]:
# extract edges by index
edge_index = torch.Tensor(edge_list[['gene1', 'gene2']].to_numpy().T).type(torch.int64)
print('edge_index shape: ', edge_index.shape)
# edge_list = torch.Tensor(np.load(edge_list_path).T).type(torch.int64) # read in format expected by pytorch geometric [2, n_edges]

# extract edge features
print()
print('edge feature columns: ', edge_feat_cols)
edge_attr = torch.Tensor(edge_list[edge_feat_cols].to_numpy())
print('edge_attr shape: ', edge_attr.shape)
# load protein-ID dictionary (need new ID system starting at index 0 for pytorch geometric)
# protein_id_dict = np.load('data/protein_ids_dict.npy', allow_pickle=True).item() # maps my custom ID system to Ensembl IDs
# protein_id_dict_inv = {Ensembl: id_ for id_, Ensembl in protein_id_dict.items()} # maps Ensembl IDs to my custom ID system

edge_index shape:  torch.Size([2, 4405641])

edge feature columns:  ['neighborhood' 'fusion' 'cooccurence' 'coexpression' 'experimental'
 'database' 'textmining' 'combined_score']
edge_attr shape:  torch.Size([4405641, 8])


In [10]:
def create_data(node_dataset, labels, label_col, test_size=0.2, val_size=0.1):
    # get subset of node features features + labels
    node_labels = node_dataset.merge(labels[label_col], left_on='myID', right_on='myID')
    
    node_feat_cols = node_labels.columns[:-1]

    X = torch.Tensor(node_labels[node_feat_cols].to_numpy())#.type(torch.float64)

    y = node_labels[label_col].fillna(-1).astype('int') # fill NaN with -1 so that it can be converted to pytorch tensor
    y = torch.Tensor(y).type(torch.int64)

    # restrict to data with labels
    node_data_labeled = node_labels[node_labels[label_col].notna()]
    train_mask, val_mask, test_mask = get_train_val_test_masks(node_data_labeled, label_col)
    
    data = Data(x=X, y=y, edge_index=edge_index, edge_attr=edge_attr)

    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask
    
    return data
    

from sklearn.model_selection import train_test_split

def get_train_val_test_masks(node_data_labeled, label_col):
    X_myIDs = node_data_labeled.index.to_numpy() # myIDs for nodes with labels for training/testing
    labels = node_data_labeled[label_col].to_numpy() # for stratification

    test_size = 0.2
    val_size = 0.1 * (1/(1-test_size))

    myIDs_train_val, myIDs_test = train_test_split(X_myIDs, test_size=test_size, shuffle=True, stratify=labels)

    labels_train_val = node_data_labeled.loc[myIDs_train_val][label_col].to_numpy()
    myIDs_train, myIDs_val = train_test_split(myIDs_train_val, test_size=val_size, shuffle=True, stratify=labels_train_val)

    # NOTE: train-val-test split is shuffled and stratified

    # create masks
    n_nodes = len(node_dataset)
    train_mask = np.zeros(n_nodes, dtype=bool)
    train_mask[myIDs_train] = True
    train_mask = torch.Tensor(train_mask).type(torch.bool)

    val_mask = np.zeros(n_nodes, dtype=bool)
    val_mask[myIDs_val] = True
    val_mask = torch.Tensor(val_mask).type(torch.bool)

    test_mask = np.zeros(n_nodes, dtype=bool)
    test_mask[myIDs_test] = True
    test_mask = torch.Tensor(test_mask).type(torch.bool)
    
    return train_mask, val_mask, test_mask

In [11]:
data = create_data(node_dataset, labels, label_col='label_0', test_size=0.2, val_size=0.1)

## Graph Convolutional Neural Network

In [12]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

# define GNN architecture
class GNNModel(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes, hidden_dense, 
                 GNN_conv_layer=GCNConv, dropout_rate=0.1, **kwargs):
        """
        Args:
            num_features (int): Dimension of input features
            hidden_channels (List[int]): Dimension of hidden features
            num_classes (int): Dimension of the output.
            hidden_dense (int): number of units in hidden dense layer following convolutions.
            GNN_conv_layer: Class of the graph convolutional layer to use.
            dropout_rate (float): Dropout rate to apply throughout the network
            kwargs: Additional arguments for the graph layer (e.g. number of heads for GAT)
        """
        super().__init__()

        self.convs = []
        self.convs.append(GNN_conv_layer(in_channels=num_features, out_channels=hidden_channels[0], **kwargs)) # first GNN Conv layer

        for c1, c2 in zip(hidden_channels[:-1], hidden_channels[1:]): # middle layers
            self.convs.append(GNN_conv_layer(in_channels=c1, out_channels=c2, **kwargs))

        self.convs = torch.nn.ModuleList(self.convs)

        self.dense1 = torch.nn.Linear(hidden_channels[-1], hidden_dense)
        self.dense_out = torch.nn.Linear(hidden_dense, num_classes)

        self.dropout_rate = dropout_rate

    def forward(self, x, edge_index):
        """
        Args:
            x: node features
            edge_index: edge list
        """

        for i,conv in enumerate(self.convs):
            x = conv(x, edge_index)
            x = x.relu()
            x = F.dropout(x, p=self.dropout_rate, training=self.training)

        x = self.dense1(x)
        x = x.relu()
        x = F.dropout(x, p=self.dropout_rate, training=self.training)
        x = self.dense_out(x)

        return x

In [13]:
# define Pytorch Lightning model
class LitGNN(pl.LightningModule):
    def __init__(self, model_name, model=None, model_type='edge', **model_kwargs):
        super().__init__()

        # Saving hyperparameters
        self.save_hyperparameters()

        self.model_name = model_name
        
        self.model_type = model_type
        if self.model_type not in ['edge', 'edge_attr', 'baseline']:
            raise TypeError("Invalid `model_type`. Must be one of ['edge', 'edge_attr', 'baseline']")
        
        # create model using GNNModel if one isn't given
        if model is None:
            self.model = GNNModel(**model_kwargs)
        else:
            self.model = model(**model_kwargs)

        # define the loss function
        self.loss_module = torch.nn.CrossEntropyLoss()

        # give example input
        self.example_input_array = data

    def forward(self, data, mode='train'):
        if self.model_type == 'edge_attr':
            x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
            x = self.model(x, edge_index, edge_attr=edge_attr)
        elif self.model_type == 'edge':
            x, edge_index = data.x, data.edge_index
            x = self.model(x, edge_index)
        else:
            x = data.x
            x = self.model(x)

        # Only calculate the loss and acc on the nodes corresponding to the mask
        if mode == 'train':
            mask = data.train_mask
        elif mode == 'val':
            mask = data.val_mask
        elif mode == 'test':
            mask = data.test_mask
        else:
            raise ValueError(f'Unknown forward mode: {mode}')

        #TODO: add other metrics like recall, precision, f1, etc...
        loss = self.loss_module(x[mask], data.y[mask])
        acc = (x[mask].argmax(dim=-1) == data.y[mask]).sum().float() / mask.sum()

        return x, loss, acc

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters())#SGD(self.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, loss, acc = self.forward(batch, mode='train')
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        logits, loss, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return logits

    def validation_epoch_end(self, validation_step_outputs):

        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        if self.logger:
            self.logger.experiment.log({'val_logits': wandb.Histogram(flattened_logits.to('cpu')), 
                                    'global_step': self.global_step})

    def test_step(self, batch, batch_idx):
        x, _, acc = self.forward(batch, mode="test")
        self.log("test_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)

In [14]:
import os
notebook_name = 'modeling_gnn.ipynb'
os.environ['WANDB_NOTEBOOK_NAME'] = notebook_name

In [15]:
from pytorch_lightning.callbacks import ModelCheckpoint, ModelSummary
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import torch_geometric.loader

from torch_geometric.nn import GCNConv, SAGEConv, GATConv, GATv2Conv, ChebConv

import datetime

In [16]:
def reset_model_params(model, verbose=False):
    '''reset model parameters'''

    for name, module in model.named_children():
        if verbose:
            print(f'resetting module {name}')
        module.reset_parameters()

from sklearn.metrics import classification_report
def evaluate_model(model, data, logger=None):
    '''returns train and test classification reports'''

    model.to(device='cuda')
    logits, _, _ = model.forward(data.to(device='cuda'))

    preds_train = logits[data.train_mask].argmax(dim=-1).cpu().detach().numpy()
    preds_test = logits[data.test_mask].argmax(dim=-1).cpu().detach().numpy()

    y_train = data.y[data.train_mask].cpu().detach().numpy()
    y_test = data.y[data.test_mask].cpu().detach().numpy()

    train_report = classification_report(y_train, preds_train, labels=[0,1], target_names=['negative', 'positive'], 
                                         output_dict=True)
    test_report = classification_report(y_test, preds_test, labels=[0,1], target_names=['negative', 'positive'], 
                                        output_dict=True)
    if logger:
        train_acc = train_report['accuracy']
        test_acc = test_report['accuracy']
        logger.log_metrics({'final_train_acc': train_acc, 'final_test_acc': test_acc})

    return train_report, test_report

def save_reports(filename, train_reports, test_reports):
    '''saves train and test reports to a json file'''
    save_dict = {'train_reports': train_reports, "test_reports": test_reports}
    json_string = json.dumps(save_dict)
    json_file = open(f'{filename}.json', 'w')
    json_file.write(json_string)
    json_file.close()


def get_value_from_dict(report, *keys):
    '''gets value from dict through sequence of keys'''
    value = report
    for key in keys:
        value = value[key]
    return value
    
def average_report_val(reports, *keys):
    '''averages a particular report value over a list of reports'''
    return np.average([get_value_from_dict(report, *keys) for report in reports])

def run_trials(model, data, start_trial=0, end_trial=100, n_epochs=500, log=False, log_project=None):

    if log:
        dt_string = str(datetime.datetime.today()).replace(' ', '_')
        if log_project is None:
            print('Enter the name of the log project: ')
            log_project = input()
    
    train_reports = []
    test_reports = []
    
    for trial in tqdm(range(start_trial, end_trial + 1)):

        print(f'running trial {str(trial)}')
        data = create_data(node_dataset, labels, f'label_{trial}', test_size=0.2, val_size=0.1)

        if log:
            n_zfills = int(np.ceil(np.log10(100)))
            log_name = f'{log_project}_trial{str(trial).zfill(n_zfills)}'

            logger = WandbLogger(name=log_name, project=log_project, log_model="all", save_dir='wandb_projects')
        else:
            logger = False

        AVAIL_GPUS = min(1, torch.cuda.device_count())

        data_loader = torch_geometric.loader.DataLoader([data], batch_size=1)

        trainer = pl.Trainer(
                    callbacks=[ModelCheckpoint(save_weights_only=False, mode="max", monitor="val_acc")],
                    #                           ModelSummary(max_depth=3)],
                    gpus=AVAIL_GPUS,
                    max_epochs=n_epochs,
                    logger=logger,
                    weights_summary=None
                    # progress_bar_refresh_rate=0,
                    )
        
        trainer.fit(model, data_loader, data_loader)
        
        model = LitGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

        train_report, test_report = evaluate_model(model, data, logger=logger)
        
        train_reports.append(train_report)
        test_reports.append(test_report)
        
        if log:
            wandb.save('modeling_gnn.ipynb')
            wandb.finish(quiet=True)
        
    return train_reports, test_reports

In [17]:
# TRAIN AND EVALUATE MODEL

model_ = torch_geometric.nn.models.GCN # base model
# model parameters
hidden_channels = 128
num_layers = 4
out_channels = num_classes

# GNN_conv_layer = GCNConv

log_project_name = 'GCN_know'

# model = LitGNN(log_project_name, model=model_, channel_list=[num_features, 128, 128, num_classes], model_type='baseline')
model = LitGNN(log_project_name, model=model_, 
                   in_channels=num_features, hidden_channels=hidden_channels,
                   num_layers=num_layers, out_channels=num_classes, model_type='edge')

# run multiple trials
train_reports, test_reports = run_trials(model, data, start_trial=0, end_trial=19, 
                                         n_epochs=500, log=True, log_project=log_project_name)

# save reports from trials to json
save_reports(f'project_reports/{log_project_name}', train_reports, test_reports)

  0%|          | 0/20 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:168: LightningDeprecationWarning: Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.
  "Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


running trial 0


wandb: Currently logged in as: awni00 (use `wandb login --relogin` to force relogin)


Validation sanity check: 0it [00:00, ?it/s]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pytorch_lightning/utilities/data.py:60: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not h

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

100%|██████████| 20/20 [1:23:23<00:00, 250.17s/it]


In [18]:
avg_acc = average_report_val(test_reports, 'accuracy')
print('Test Accuracy: ', avg_acc)

avg_f1 = average_report_val(test_reports, 'weighted avg', 'f1-score')
print('Test f1-score: ', avg_f1)

Test Accuracy:  0.7628205128205129
Test f1-score:  0.7617568621926686


In [19]:
# clear local log files that have been synced to wandb
#!printf 'y\n' | wandb sync --clean